# Temporal Graph Analysis

## Prerequisites

First, we need to set up our Python environment that has PyTorch, PyTorch Geometric and PathpyG installed. Depending on where you are executing this notebook, this might already be (partially) done. E.g. Google Colab has PyTorch installed by default so we only need to install the remaining dependencies. The DevContainer that is part of our GitHub Repository on the other hand already has all of the necessary dependencies installed. 

In the following, we install the packages for usage in Google Colab using Jupyter magic commands. For other environments comment in or out the commands as necessary. For more details on how to install `pathpyG` especially if you want to install it with GPU-support, we refer to our [documentation](https://www.pathpy.net/dev/getting_started/). Note that `%%capture` discards the full output of the cell to not clutter this tutorial with unnecessary installation details. If you want to print the output, you can comment `%%capture` out.

In [ ]:
%%capture
# !pip install torch
!pip install torch_geometric
!pip install git+https://github.com/pathpy/pathpyG.git

## Motivation and Learning Objectives

In this tutorial we will introduce the representation of temporal graph data using the `TemporalGraph` class and how such data can be used to calculate shortest time respecting paths between nodes as well temporal node cemtralities.

In [1]:
import torch
from torch_geometric.data import TemporalData
import pathpyG as pp

pp.config['torch']['device'] = 'cpu'

We can create a temporal graph object from a list of time-stamped edges. Since `TemporalGraph` is a subclass of the `Graph` class, the internal structures are very similar:

In [2]:
tedges = [('a', 'b', 1),('a', 'b', 2), ('b', 'a', 3), ('b', 'c', 3), ('d', 'c', 4), ('a', 'b', 4), ('c', 'b', 4),
              ('c', 'd', 5), ('b', 'a', 5), ('c', 'b', 6)]
t = pp.TemporalGraph.from_edge_list(tedges)
print(t.mapping)
print(t.N)
print(t.M)

a -> 0
b -> 1
c -> 2
d -> 3

4
10


By default, all temporal graphs are directed. We can create an undirected version a temporal graph as follows:

In [ ]:
x = t.to_undirected()
print(x.mapping)
print(x.N)
print(x.M)

We can also directly create a temporal graph from an instance of `pyG.TemporalData`

In [ ]:
td = TemporalData(
    src = torch.Tensor([0,1,2,0]),
    dst = torch.Tensor([1,2,3,1]), 
    t = torch.Tensor([0,1,2,3]))
print(td)
t2 = pp.TemporalGraph(td)
print(t2)

We can restrict a temporal graph to a time window, which returns a temporal graph that only contains time-stamped edges in the given time interval.

In [ ]:
t1 = t.get_window(0,4)
print(t1)
print(t1.start_time)
print(t1.end_time)

We can easily convert a temporal graph into a weighted time-aggregated static graph, where edge weights count the number of occurrences of an edge across all timestamps.

In [ ]:
g = t.to_static_graph(weighted=True)
print(g)

We can also aggregate a temporal graph within a certain time window:

In [ ]:
g = t.to_static_graph(time_window=(1, 3), weighted=True)
print(g)

Finally, we can use the class `RollingTimeWindow` to perform a rolling window analysis. The class returns an iterable object, where each iteration yields a time-aggregated weighted graph object as well as the corresponding time window.

In [ ]:
r = pp.algorithms.RollingTimeWindow(t, window_size=3, step_size=1, return_window=True)
for g, w in r:
    print('Time window ', w)
    print(g)
    print(g.data.edge_index)
    print('---')

We can visualize temporal graphs using the plot function just like static graphs:

In [ ]:
pp.plot(t, node_label=t.mapping.node_ids.tolist(), edge_color='lightgray');

The source nodes, destination nodes and timestamps of time-stamped edges are stored as a `pyG TemporalData` object, which we can access in the following way.

In [ ]:
t.data

In [ ]:
print(t.data.edge_index)

In [ ]:
print(t.data.t)

With the generator functions `edges` and `temporal_edges` we can iterate through the time-ordered (temporal) multi-edges of a temporal graph.

In [ ]:
for v, w in t.edges:
    print(v, w)

In [ ]:
for v, w, time in t.temporal_edges:
    print(v, w, time)

## Extracting Time-Respecting Paths in Temporal Networks

We are often interested in time-respecting paths in a temporal graph. A time-respecting path consists of a sequence of nodes $v_0,...,v_l$ where consecutive nodes are connected by time-stamped edges that occur (i) in the right temporal ordering, and (ii) within a maximum time difference of $\delta\in \N$. 

To calculate time-respecting paths in a temporal graph, we can construct a directed acyclic graph (DAG), where each time-stamped edge $(u,v;t)$ in the temporal graph is represented by a node and two nodes representing time-stamped edges $(u,v;t_1)$ and $(v,w;t_2)$ are connected by an edge iff $0 < t_2-t_1 \leq \delta$. This implies that (i) each edge in the resulting DAG represents a time-respecting path of length two, and (ii) time-respecting paths of any lenghts are represented by paths in this DAG.

We can construct such a DAG using the function `pp.algorithms.lift_order_temporal`, which returns an edge_index. We can pass this to the constructor of a `Graph` object, which we can use to visualize the resulting DAG.

In [ ]:
e_i = pp.algorithms.lift_order_temporal(t, delta=1)
dag = pp.Graph.from_edge_index(e_i)
pp.plot(dag, node_label = [f'{v}-{w}-{time}' for v, w, time in t.temporal_edges]);

For $\delta=1$, this DAG with three connected components tells us that the underlying temporal graph has  the following time-respecting paths (of different lengths):

Length one:  
    a -> b  
    b -> a  
    b -> c  
    c -> b  
    c -> d  
    d -> c  

Length two:  
    a -> b -> a (twice, starting at time 2 and time 4)  
    b -> a -> b  
    a -> b -> c     
    b -> c -> b  
    c -> b -> a  
    d -> c -> d  

Length three:   
    a -> b -> a -> b  
    b -> a -> b -> a  
    a -> b -> c -> b  
    b -> c -> b -> a  
    
Length four:   
    a -> b -> a -> b -> a  
    a -> b -> c -> b -> a  

We can can use the function `pp.algorithms.temporal.temporal_shortest_paths` to calculate shortest time-respecting path distances between any pair of nodes. This also returns a predecessor matrix, which can be used to reconstruct all shortest time-respecting paths (in analogy to the Dijkstra algorithm for static graphs):

In [ ]:
dist, pred = pp.algorithms.temporal.temporal_shortest_paths(t, delta=1)
print(t.mapping)
print(dist)
print(pred)

In the example above, the four `inf` values indicate that there is no time-respecting paths between the four node pairs (a, d), (b, d), (d,a) and (d, b). This is not something we would expect based on the (strongly connected) topology of the time-aggregated graph, which is shown below:

In [ ]:
g = t.to_static_graph(weighted=True)
pp.plot(g, node_label=g.mapping.node_ids.tolist());

## Reading and writing temporal graph data

In [3]:
tedges = [('a', 'b', 1),('a', 'b', 2), ('b', 'a', 3), ('b', 'c', 3), ('d', 'c', 4), ('a', 'b', 4), ('c', 'b', 4),
              ('c', 'd', 5), ('b', 'a', 5), ('c', 'b', 6)]
t = pp.TemporalGraph.from_edge_list(tedges)
df = pp.io.to_df(t)
print(df)

   v  w    t
0  c  b  6.0
1  b  a  5.0
2  c  d  5.0
3  c  b  4.0
4  a  b  4.0
5  d  c  4.0
6  b  c  3.0
7  b  a  3.0
8  a  b  2.0
9  a  b  1.0


In [4]:
t = pp.io.df_to_temporal_graph(df)
print(t)

Temporal Graph with 4 nodes, 6 unique edges and 10 events in [1.0, 6.0]

Graph attributes
	dst		<class 'torch.Tensor'> -> torch.Size([10])
	src		<class 'torch.Tensor'> -> torch.Size([10])
	t		<class 'torch.Tensor'> -> torch.Size([10])



/opt/conda/lib/python3.10/site-packages/torch_geometric/data/storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'dst', 'src', 't'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


In [5]:
pp.io.write_csv(t, '../data/test_temporal_graph.csv')

In [11]:
t = pp.io.read_csv('../data/test_temporal_graph.csv')
print(t)

Temporal Graph with 4 nodes, 6 unique edges and 10 events in [1.0, 6.0]

Graph attributes
	dst		<class 'torch.Tensor'> -> torch.Size([10])
	src		<class 'torch.Tensor'> -> torch.Size([10])
	t		<class 'torch.Tensor'> -> torch.Size([10])



/opt/conda/lib/python3.10/site-packages/torch_geometric/data/storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'dst', 'src', 't'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


## Temporal Centralities in Empirical Temporal Networks

`pathpyG`'s ability to calculate (shortest) time-respecting paths enables us to calulate different notions of temporal centralities for nodes in empirial temporal networks. We can read an empirical temporal graph based on CSV data, where each line contains the source, target, and timestamp of an edge as comma-separated value:

In [9]:
t_ants = pp.io.read_csv('../data/ants_1_1.tedges')
print(t_ants)

Temporal Graph with 89 nodes, 947 unique edges and 1911 events in [0.0, 1438.0]

Graph attributes
	dst		<class 'torch.Tensor'> -> torch.Size([1911])
	src		<class 'torch.Tensor'> -> torch.Size([1911])
	t		<class 'torch.Tensor'> -> torch.Size([1911])



/opt/conda/lib/python3.10/site-packages/torch_geometric/data/storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'dst', 'src', 't'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


To calculate the temporal closeness centrality, which is defined based on the length of shortest time-respecting paths of a node to all other nodes, we can write the following: 

In [10]:
cl = pp.algorithms.centrality.temporal_closeness_centrality(t_ants, delta=60)
print(cl)
mx = max(cl.values())
mn = min(cl.values())
node_size = { v: 50*(x/(mx-mn)) for v, x in cl.items() }
pp.plot(t_ants, node_size=node_size, edge_color='red', edge_size=4);

100%|██████████| 883/883 [00:00<00:00, 2486.64it/s]


{'Y_WY': 3116.980952380952, 'WRBB': 3178.1565323565333, 'WRR_': 1176.2666666666669, 'GGRR': 3290.5015873015864, 'WG_R': 3482.3904761904764, 'YWGW': 2530.0349206349183, 'YYGG': 3001.533333333333, 'GBGR': 1855.368253968255, 'GRWG': 2547.6000000000004, 'YY_W': 2677.6095238095236, 'G_W_': 2350.5724053724057, 'G___small': 1365.7142857142862, 'YW__': 176.0, 'YYGGmid': 3818.571428571429, '____brood': 2413.399999999999, 'GGWY': 4004.9111111111115, 'G_R_': 2072.0857142857144, '____topleft': 2619.606349206348, 'G_GW': 2843.2380952380945, 'GYYY': 1278.2380952380954, 'WBGW': 2560.2095238095235, 'Y__W': 1926.749206349207, 'YYYY': 1377.2454212454215, 'GWRG': 2755.4825396825395, 'YGWW': 3590.1904761904752, 'WRWR': 2932.0761904761894, 'G___big': 909.2317460317461, 'GY__': 1401.8190476190475, 'WRRY': 3007.5047619047614, 'GR_Y': 536.1714285714286, 'YWW_': 2506.219047619047, '_W__': 3150.2253968253954, '__W_': 2607.7682539682523, 'WBYG': 1891.8634920634915, 'YGWY': 2095.7619047619037, '_R__': 3534.876190

The definition of time-respecting paths depends on our maximum time difference parameter $\delta$, which implies that different values of this parameter also yield different centralities. This means that we can calculate temporal node centralities for different "time scales" of a temporal graph.

In [ ]:
cl = pp.algorithms.centrality.temporal_closeness_centrality(t_ants, delta=20)
print(cl)
mx = max(cl.values())
mn = min(cl.values())
node_size = { v: 50*(x/(mx-mn)) for v, x in cl.items() }
pp.plot(t_ants, node_size=node_size, edge_color='red', edge_size=4);

We can also calculate the temporal betweenness centrality, which is based on the number of shortest time-respecting paths between pairs of nodes that pass through a given node. Again, this centrality score is sensitive to the time scale parameter $\delta$.

In [ ]:
bw = pp.algorithms.centrality.temporal_betweenness_centrality(t_ants, delta=60)
print(bw)
mx = max(bw.values())
mn = min(bw.values())
node_size = { v: 50*(x/(mx-mn)) for v, x in bw.items() }
pp.plot(t_ants, node_size=node_size, edge_color='red', edge_size=4);

In [ ]:
bw = pp.algorithms.centrality.temporal_betweenness_centrality(t_ants, delta=20)
print(bw)
mx = max(bw.values())
mn = min(bw.values())
node_size = { v: 50*(x/(mx-mn)) for v, x in bw.items() }
pp.plot(t_ants, node_size=node_size, edge_color='red', edge_size=4);